# Conversation Analytics

Analyze conversation patterns, performance metrics, and user interactions from the Voice AI Agent.

## Metrics covered:
- Session statistics (duration, turns, completion rate)
- Intent distribution
- Voice processing performance (STT/TTS latency)
- Conversation flow patterns

In [ ]:
# Setup
import sys
sys.path.insert(0, '/app')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Create database connection
engine = create_engine(os.getenv('DATABASE_URL'))

print("✓ Analytics notebook ready")

## 1. Session Overview

In [ ]:
# Get session statistics
query = """
SELECT 
    DATE(created_at) as date,
    COUNT(*) as total_sessions,
    COUNT(CASE WHEN status = 'completed' THEN 1 END) as completed_sessions,
    COUNT(CASE WHEN status = 'active' THEN 1 END) as active_sessions,
    COUNT(CASE WHEN direction = 'inbound' THEN 1 END) as inbound_calls,
    COUNT(CASE WHEN direction = 'outbound' THEN 1 END) as outbound_calls
FROM conversations.conversation_sessions
WHERE created_at >= NOW() - INTERVAL '30 days'
GROUP BY DATE(created_at)
ORDER BY date DESC;
"""

df_sessions = pd.read_sql(query, engine)
print(f"Sessions in last 30 days: {df_sessions['total_sessions'].sum()}")
display(df_sessions.head(10))

In [ ]:
# Visualize session trends
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Session volume over time
axes[0].plot(df_sessions['date'], df_sessions['total_sessions'], marker='o', linewidth=2, label='Total')
axes[0].plot(df_sessions['date'], df_sessions['completed_sessions'], marker='s', linewidth=2, label='Completed')
axes[0].set_title('Session Volume Over Time', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Number of Sessions')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Inbound vs Outbound
axes[1].bar(df_sessions['date'], df_sessions['inbound_calls'], label='Inbound', alpha=0.7)
axes[1].bar(df_sessions['date'], df_sessions['outbound_calls'], bottom=df_sessions['inbound_calls'], 
           label='Outbound', alpha=0.7)
axes[1].set_title('Call Direction Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Number of Calls')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Conversation Turn Analysis

In [ ]:
# Get conversation turn statistics
query = """
SELECT 
    cs.session_id,
    cs.session_type,
    cs.status,
    COUNT(ct.turn_id) as total_turns,
    AVG(ct.processing_time) as avg_processing_time,
    cs.created_at,
    cs.ended_at,
    EXTRACT(EPOCH FROM (cs.ended_at - cs.created_at)) as duration_seconds
FROM conversations.conversation_sessions cs
LEFT JOIN conversations.conversation_turns ct ON cs.session_id = ct.session_id
WHERE cs.created_at >= NOW() - INTERVAL '7 days'
GROUP BY cs.session_id, cs.session_type, cs.status, cs.created_at, cs.ended_at;
"""

df_turns = pd.read_sql(query, engine)
print(f"Conversations analyzed: {len(df_turns)}")
print(f"\nAverage turns per conversation: {df_turns['total_turns'].mean():.1f}")
print(f"Average conversation duration: {df_turns['duration_seconds'].mean():.1f} seconds")
display(df_turns.head())

In [ ]:
# Visualize turn distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Turn count distribution
axes[0].hist(df_turns['total_turns'].dropna(), bins=20, edgecolor='black', alpha=0.7)
axes[0].axvline(df_turns['total_turns'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df_turns["total_turns"].mean():.1f}')
axes[0].set_title('Distribution of Turns per Conversation', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Number of Turns')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Duration distribution
axes[1].hist(df_turns['duration_seconds'].dropna()/60, bins=20, edgecolor='black', alpha=0.7, color='green')
axes[1].axvline(df_turns['duration_seconds'].mean()/60, color='red', linestyle='--',
                label=f'Mean: {df_turns["duration_seconds"].mean()/60:.1f} min')
axes[1].set_title('Conversation Duration Distribution', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Duration (minutes)')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Voice Processing Performance

In [ ]:
# Get STT/TTS performance metrics
query_stt = """
SELECT 
    DATE_TRUNC('hour', created_at) as hour,
    COUNT(*) as total_transcriptions,
    AVG(processing_time) as avg_processing_time,
    AVG(audio_duration) as avg_audio_duration,
    AVG(confidence_score) as avg_confidence
FROM voice_data.transcriptions
WHERE created_at >= NOW() - INTERVAL '7 days'
GROUP BY DATE_TRUNC('hour', created_at)
ORDER BY hour DESC;
"""

query_tts = """
SELECT 
    DATE_TRUNC('hour', created_at) as hour,
    COUNT(*) as total_syntheses,
    AVG(processing_time) as avg_processing_time,
    AVG(character_count) as avg_characters
FROM voice_data.syntheses
WHERE created_at >= NOW() - INTERVAL '7 days'
GROUP BY DATE_TRUNC('hour', created_at)
ORDER BY hour DESC;
"""

try:
    df_stt = pd.read_sql(query_stt, engine)
    df_tts = pd.read_sql(query_tts, engine)
    
    print(f"STT operations analyzed: {df_stt['total_transcriptions'].sum()}")
    print(f"TTS operations analyzed: {df_tts['total_syntheses'].sum()}")
    print(f"\nAverage STT processing time: {df_stt['avg_processing_time'].mean():.3f}s")
    print(f"Average TTS processing time: {df_tts['avg_processing_time'].mean():.3f}s")
except Exception as e:
    print(f"Note: Voice processing tables may be empty. Error: {e}")

## 4. Intent Classification Analysis

In [ ]:
# Get intent distribution from conversation turns
query = """
SELECT 
    intent,
    COUNT(*) as count,
    AVG(confidence_score) as avg_confidence
FROM conversations.conversation_turns
WHERE created_at >= NOW() - INTERVAL '7 days'
  AND intent IS NOT NULL
GROUP BY intent
ORDER BY count DESC
LIMIT 10;
"""

try:
    df_intents = pd.read_sql(query, engine)
    
    if len(df_intents) > 0:
        # Create pie chart
        plt.figure(figsize=(10, 6))
        plt.pie(df_intents['count'], labels=df_intents['intent'], autopct='%1.1f%%', startangle=90)
        plt.title('Top 10 Intent Distribution (Last 7 Days)', fontsize=14, fontweight='bold')
        plt.axis('equal')
        plt.show()
        
        display(df_intents)
    else:
        print("No intent data available yet")
except Exception as e:
    print(f"Intent analysis not available: {e}")

## 5. Daily Analytics Summary

In [ ]:
# Get daily summary from analytics schema
query = """
SELECT 
    date,
    total_sessions,
    total_turns,
    avg_session_duration,
    avg_turns_per_session,
    total_transcriptions,
    total_syntheses
FROM analytics.daily_stats
ORDER BY date DESC
LIMIT 30;
"""

try:
    df_daily = pd.read_sql(query, engine)
    
    if len(df_daily) > 0:
        print("Daily Analytics Summary (Last 30 Days)")
        display(df_daily)
    else:
        print("No daily analytics data available yet")
except Exception as e:
    print(f"Daily analytics not available: {e}")

## Custom Queries

Use this section for your own custom analysis queries.

In [ ]:
# Your custom query here
custom_query = """
-- Add your SQL query
SELECT 1;
"""

# df_custom = pd.read_sql(custom_query, engine)
# display(df_custom)